# "Gather" Data Containing Robert Parker Critic Scores

In [97]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [98]:
df = pd.read_csv('data/all_jera_info.csv')
df['producer'] = (df['producer_title'].fillna('') + ' ' + df['producer_name']).str.strip()
producer_list = df['producer'].unique().tolist()

In [99]:
def reestablish_connection(url):
    
    # Setup driver
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.get(url)
    try:
        WebDriverWait(driver, 12).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#didomi-notice-agree-button"))
    ).click()
    except:
        print("No cookie popup")

    # "callum@winefi.co" "Testing!12" -- If large screen (chrome)
    wait = WebDriverWait(driver, 10)
    login_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='btn btn-default'][span='LOGIN']")))
    login_button.click()

    email_input = driver.find_element(By.ID, "user_login")
    email_input.send_keys("callum@winefi.co")
    time.sleep(3)

    password_input = driver.find_element(By.ID, "user_pass")
    password_input.send_keys("Testing!12")
    time.sleep(1)

    submit_button = driver.find_element(By.ID, "submit-login")
    submit_button.click()

    return driver

In [101]:
"""
This function takes a producer name and returns only the first 3 words unless the first three are 
'Domaine de la', in which case it returns the first 4 words. 

It is also the function used to clean up unique cases where Robert Parker website had a variation in name. 
"""
def producer_filter(producer):
    words = producer.split()
    if producer == 'Domaine Anne-Francoise Gros':
        return 'Domaine Anne Gros'
    if producer == 'Jean-Claude Ramonet':
        return 'Domaine Ramonet'
    if producer == 'Domaine Francois Raveneau':
        return 'Domaine Raveneau'
    if producer == 'Domaine Dauvissat-Camus':
        return 'Vincent Dauvissat'
    if producer == 'Domaine Bournet-Lapostolle':
        return 'Lapostolle'
    if producer == 'Chateau Rayas Fonsalette':
        return 'Fonsalette'
    if producer == 'Domaine Francois Lamarche' or producer == 'Francois Lamarche':
        return "Domaine Lamarche"
    if words[0] == 'Chateau':
        return' '.join(words[1:]).strip()
    if words[0] == 'Domaine':
        return' '.join(words[1:]).strip()
    elif producer == 'Comm. G.B. Burlotto':
        return 'Comm. '
    elif producer == "Dow's":
        return "Dow"
    elif producer == "Taylor's":
        return "Taylor"
    elif producer == 'Moret Nominee':
        return "David Moret"
    elif producer == 'Dom Perignon':
        return "Moet & Chandon"
    elif producer == 'R. Lopez de Heredia':
        return 'Lopez de Heredia'
    elif producer == 'The Sadie Family':
        return 'Sadie Family'
    elif producer == 'R. Lopez de Heredia':
        return 'Lopez de Heredia'
    elif producer ==  'Oratoire Chanson':
        return 'Chanson Père & Fils'
    elif len(producer.split()) > 3:
        words = producer.split()
        first_two_words = ' '.join(words[:2])
        if first_two_words.lower() == 'domaine de':
            if ' '.join(words[:2]).lower() == 'domaine de la':
                return ' '.join(words[:4])
            return ' '.join(words[:3])
        else:
            return first_two_words
    else:
        return producer

In [102]:
# BEST ONE 3/5/21
def search_and_extract(driver, producer_name_input, searchnum=6):
        
    producer_name = producer_filter(producer_name_input)

    search_input = driver.find_element(By.ID, f"autocomplete-{searchnum}-input")
    time.sleep(0.5)
    search_input.send_keys(producer_name)
    time.sleep(2.1)

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, f"autocomplete-{searchnum}-0-item-0"))
    ).click()

    wine_data = []
    
    while True:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.wine-check"))
        )

        wines = driver.find_elements(By.CSS_SELECTOR, "div.wine-check")

        for wine in wines:
            description = wine.find_element(By.CSS_SELECTOR, "label").get_attribute('title')
            full_name_element = wine.find_element(By.CSS_SELECTOR, "a.no-style.font-weight-bold")
            full_name = full_name_element.text
            product_name = wine.find_element(By.CSS_SELECTOR, "input").get_attribute('name')
            score_element = wine.find_element(By.XPATH, "//div[@class='wine-ratings']//span[contains(text(), 'RP')]")
            critic_score = score_element.text
            wine_data.append({
                "producer_name": producer_name_input,
                "product_name": product_name,
                "description": description,
                "full_name": full_name,
                "critic_score": critic_score
            })

        try:
            next_button = driver.find_element(By.XPATH, "//span[contains(@class,'sc-jtXEFf') and contains(text(),'>')]")
            next_button.click()
            time.sleep(1.5)
        except:
            break

    return wine_data


In [103]:
# Scrape by producer name list function 
def loop_search_extract_producers(url, producer_list, csv_path = 'output.csv', snum=6):
 
    output_list = []
    skipped_producer_list = []
    driver = reestablish_connection(url)

    for producer in producer_list:
        try:
            extracted_data = search_and_extract(driver, producer, snum)
            output_list.extend(extracted_data)
            snum += 1
        except Exception as e:
            print(f"Error with producer {producer}: {e}")
            skipped_producer_list.append(producer)
            # Reestablish connection before continuing - login etc.
            driver.quit()
            time.sleep(0.6)
            driver = reestablish_connection(url)
            snum = 6
            time.sleep(3)

    driver.quit()

    print(f"Failed producers:{len(skipped_producer_list)}")
    print(f"Successful wines scrapes:{len(output_list)}")

    output_list = pd.DataFrame(output_list, columns=['producer_name', 'product_name', 
                                                        'description', 'full_name', 'critic_score'])

    output_list.to_csv(csv_path, index = False)
    print(f"NaNs in scrape{output_list.isna().sum()}")
    return output_list, skipped_producer_list

final_prods = ['Domaine Dauvissat-Camus',
 'Domaine Francois Lamarche',
 'Domaine Bournet-Lapostolle']

s6_list, s6_skipped_prods = loop_search_extract_producers('https://www.robertparker.com/', final_prods, csv_path='full_scrape_6.csv')

Failed producers:0
Successful wines scrapes:350
NaNs in scrapeproducer_name    0
product_name     0
description      0
full_name        0
critic_score     0
dtype: int64


In [ ]:
# Combine all full scrape csvs into one csv and export 
s1 = pd.read_csv('def_done/full_scrape_1.csv')
s2 = pd.read_csv('def_done/full_scrape_2.csv')
s3 = pd.read_csv('def_done/full_scrape_3.csv')
s4 = pd.read_csv('def_done/full_scrape_4.csv')
s5 = pd.read_csv('def_done/full_scrape_5.csv')
s6 = pd.read_csv('def_done/full_scrape_6.csv')

# combine all csvs into one
master_list = pd.concat([s1, s2, s3, s4, s5, s6], ignore_index=True)
master_list.drop(columns = 'Unnamed: 0', inplace=True)
# drop duplicates rows in master_list
master_list.drop_duplicates(inplace=True)
master_list.reset_index(drop=True, inplace=True)
master_list.to_csv('def_done/master_full_scrape.csv', index = False)

In [114]:
master_list['producer_name'].nunique()

260

**Scraping project results:**
- Sucessfully scraped scores for 33,533 wines covering 260 producers. 
- Unable to find scores for only one of the producers in our current dataset (Jera) - namelu NATHALIE VIGOT, a very rare burgundy producer (3500 bottles a year).
- Code is ready and waiting to be used on additional producers when we expand the dataset.

**Next steps:**
- Attempt to match wines with critic scores, scraped from RP, with LWIN11s in our current dataset. This should be easily doable for all those producers within our current dataset that only have 1 LWIN7, about half of the producers.
- For the remainder I plan to use Wine Compare match API and have contacted O

## Spare Code

In [15]:
# # Test code 
# test_1 = search_and_extract('Keller')
# time.sleep(2)
# test_2 = search_and_extract('Domaine Leroy', searchnum=7)
# test_combined = []
# test_combined = test_combined + test_1
# test_combined = test_combined + test_2
# test_combined = pd.DataFrame(test_combined, columns=['producer_name', 'product_name', 
#                                                      'description', 'full_name', 'critic_score'])

In [29]:
# snum = 6
# driver = reestablish_connection('https://www.robertparker.com/')
# extracted_data = search_and_extract(driver, 'Keller', searchnum=snum)
# driver.quit()

In [27]:
# first_time = True
# combined_lists = []
# failed_producers = []
# for producer in producer_list:
#     if first_time == True:
#         try:
#             prod_list = search_and_extract(producer)
#         except:
#             print(f"Unable to extract for {producer}")
#             failed_producers += producer 
#         first_time = False
#     else:
#         if producer == 'Chiara Boschis (Azienda Agricola E. Pira e Figli)':
#             producer = 'Chiara Boschis'
#         try:
#             prod_list = search_and_extract(producer, firsttime=False)
#         except:
#             print(f"Unable to extract for {producer}")
#             failed_producers += producer 

#     combined_lists += prod_list
#     time.sleep(1)

In [28]:
# test = search_and_extract('Domaine Leroy')

In [29]:
# producer_names = ["Producer1", "Producer2"]  # Add as many producer names as you want
# all_data = []

# for name in producer_names:
#     all_data.extend(search_and_extract(name))

# df = DataFrame(all_data, columns=['Producer Name', 'Product Name', 'Critic Score'])
# df.to_csv('output.csv', index=False)


In [30]:
# def search_and_extract(producer_name):

#     search_input = driver.find_element(By.ID, "autocomplete-6-input")

#     search_input.send_keys(producer_name)


#     # Wait and click the first dropdown result
#     WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.ID, "autocomplete-6-0-item-0"))
#     ).click()

#     wine_data = []

#     while True:
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.wine-check"))
#         )

#         wines = driver.find_elements(By.CSS_SELECTOR, "div.wine-check")

#         for wine in wines:
#             description = wine.find_element(By.CSS_SELECTOR, "label").get_attribute('title')
#             full_name_element = wine.find_element(By.CSS_SELECTOR, "a.no-style.font-weight-bold")
#             full_name = full_name_element.text
#             product_name = full_name_element.get_attribute("name")
            
#             # Extracting critic score using BeautifulSoup
#             html_content = wine.get_attribute('innerHTML')
#             soup = BeautifulSoup(html_content, 'html.parser')
#             score_element = soup.find('div', class_="sc-bhnkmi elYHdv ratings").find('span')
#             critic_score = score_element.text
#             wine_data.append({
#                 "producer_name": producer_name,
#                 "product_name": product_name,
#                 "description": description,
#                 "full_name": full_name,
#                 "critic_score": critic_score
#             })

#         try:
#             next_button = driver.find_element(By.XPATH, "//span[contains(@class,'sc-jtXEFf') and contains(text(),'>')]")
#             next_button.click()
#         except:
#             break
# #test = search_and_extract('Domaine Leroy')